In [1]:
import torch
import asyncio
from sentence_transformers import SentenceTransformer
from abc import ABC,abstractclassmethod

class BaseEmbedding(ABC):

    @abstractclassmethod
    def _generate_embeddings(self, text):
        pass
    @abstractclassmethod
    def embed(self, text):
        pass


class SentenceTransformersEmbedding(BaseEmbedding):
    def __init__(self, model_name:str = "sentence-transformers/all-MiniLM-L6-v2", device:str = 'cpu',show_progress:str = False):
        super().__init__()
        self.show_progress = show_progress
        self.model_name = model_name
        self.device = device
        self.model = SentenceTransformer(self.model_name,device=self.device)

    def _generate_embeddings(self, text):
        embeddings = self.model.encode(text,show_progress_bar=self.show_progress).tolist()
        return embeddings
        
    def embed(self, text):
        return self._generate_embeddings(text)

In [4]:
import os
from sentence_transformers import SentenceTransformer
import chromadb
from semantic_text_splitter import HuggingFaceTextSplitter
from tokenizers import Tokenizer
from chromadb import Documents, EmbeddingFunction, Embeddings

class ChromaDB:
    def __init__(self,embedding_model = None):
        if embedding_model == None:
            self.embedder = SentenceTransformersEmbedding(device = "cpu")
        else:
            self.embedder = embedding_model

    def save(self,documents, db_path,collection_name:str = "quickstart" ):
        client = chromadb.PersistentClient(path=os.path.abspath(db_path))
        collection = client.create_collection(collection_name,metadata={"hnsw:space": "cosine"})
        embeddings = self._get_document_embeddings(documents)
        collection.add(embeddings=embeddings, documents=documents, ids=[f"id{i}" for i in range(1, len(embeddings) + 1)])
        print(f"ChromaDB saved successfully to {db_path}")
        
    def _get_document_embeddings(self,text):
        return self.embedder.embed(text)
        
    def retrieve(self,user_input,db_path,collection_name = "quickstart",top_k = 1):
        client = chromadb.PersistentClient(path=os.path.abspath(db_path))
        collection = client.get_collection(collection_name)
        # return collection  
        results = collection.query(
        query_embeddings = [self._get_document_embeddings(user_input)],
        n_results=top_k,
        )
        scores = results["distances"][0]
        documents = results["documents"][0]
        ids = results["ids"][0]
        retrieved_data = [[document,score,id] for document,score,id in zip(documents, scores,ids)]
        return retrieved_data


In [5]:
from semantic_text_splitter import HuggingFaceTextSplitter
def segment_text(file_contents, chunk_size):
        tokenizer = Tokenizer.from_pretrained("bert-base-uncased")
        splitter = HuggingFaceTextSplitter(tokenizer, trim_chunks=False)

        sentences = splitter.chunks(file_contents, chunk_size)

        result_chunks = []
        current_chunk = ""
        for i in range(len(sentences)):
            current_chunk += f" {sentences[i]}"
            if current_chunk.count(" ") >= 256:
                result_chunks.append(current_chunk)
                current_chunk = f"{sentences[i-2]} {sentences[i-1]} {sentences[i]}"
        return result_chunks

In [6]:
with open("grand_cholan.txt") as f:
    text = f.read()

text = segment_text(text,chunk_size=50)

In [7]:
embed_model = SentenceTransformersEmbedding(model_name="BAAI/bge-large-en")

/home/vectone/anaconda3/envs/myenv/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [8]:
db = ChromaDB(embedding_model=embed_model)
db.save(text,"./text","bharathi2001")

UniqueConstraintError: Collection bharathi2001 already exists

In [9]:
db.retrieve("price of idly","./text","bharathi2001")

[['Tandoori Chicken (Half)\n£12.50\n(Chicken on the bones marinated and cooked in clay oven)\nLamb Chops\n£10.95\n(Lamb chops marinated with chef’s special spices & cooked in clay oven)\n Salmon Tikka\n£11.95\n(Fresh Salmon marinated with chef’s special spices & cooked in clay oven)\nTandoori King Prawns Tikka\n£12.95\n (Large King Prawns marinated with chef’s special tandoori spices and grilled in clay oven)\nIndian Vegetable Curry\nBaingan Bharta\n£12.95\n (Fresh aubergine grilled in a clay oven and prepared with onion tomato sauce)\nBhindi Masala\n£12.95\n(Ladyfingers/cut okra sautéed with bell peppers, tomato and onion)\n Vegetable Chettinad Curry\n£12.95\n(Fresh Vegetables curry cooked with South Indian special spices)\nEgg Masala\n£12.95\n(Egg cooked with onion, tomato and South Indian spices)\nBombay Aloo\n£9.95\n (Baby potatoes fried and tossed in Indian spices)\nAloo Gobi\n£10.50\n(Baby potatoes & cauliflower cooked in Indian spices)\nRasam (South Indian Soup)\n£6.95\n (Boost 

In [35]:
# from pydantic import List
# class Ensemble:

#     def __init__(self,vector_db,model_names:List[str]):
#         self.vector_db = vector_db
#         self.model_names = model_names

#     def generate_batch():
#         for i in range(1,6):
#             yield i
        
#     def save(self,):
#         # Initialize models and store in a list
#         models = [CustomEmbeddingModel(model_name=name) for name in model_names]
#         embeddings = model.encode_text(segmented_text)
#         database_path = f"./embedded_v2/path2_{no}"
#         model.save_vectors_to_chromadb(embeddings, segmented_text, database_path)
            

In [10]:
import requests
import re
def generate_queries(original_query):
    payload = {
      "question": original_query,
      "company": "grandcholan",
      "domain": "foods and services"
    }
    
    response = requests.post("http://46.43.144.145:7030/variation_rag",json = payload)
    
    variation_text = response.json()
    
    pattern = re.compile(r"^([^\?]+)(\?.*)?$")
    generated_queries=[]
   
    for line in variation_text.splitlines():
        match = pattern.search(line)
        if match:
            question = match.group(1)
            generated_queries.append(question.strip()[3:])
    
    return generated_queries

In [11]:
generate_queries("price of idli")

['Cost of idli',
 "Idli's price range",
 'How much does an idli cost',
 'What is the value of one idli']

In [31]:
def get_context(query):
    query_variations = generate_queries(query)+[query]
    print(query_variations)
    before_reranking = []
    for que in query_variations:
        single_query_context = []
        context = db.retrieve(que,"./text","bharathi1604",top_k=3)
        for i in context:
            single_query_context.append({"id":i[2],"text":i[0]})
        before_reranking.extend(single_query_context)
    return before_reranking
        
        

In [32]:
before_reranking = get_context(query)


['How to reserve an airplane ticket', 'Booking a plane reservation', 'Purchasing an air travel ticket', 'Obtaining flight tickets', 'how to book a flight ticket']


In [33]:
from flashrank.Ranker import Ranker, RerankRequest
# Nano (~4MB), blazing fast model & competitive performance (ranking precision).
# ranker = Ranker()
# ranker = Ranker(model_name="rank-T5-flan")
ranker = Ranker(model_name="ms-marco-MultiBERT-L-12")

In [34]:
def rerank_result(query):
    before_reranking = get_context(query)
    rerankrequest = RerankRequest(query, passages=before_reranking)
    results = ranker.rerank(rerankrequest)
    final_context = []
    for result in results:
        # print(result,"\n\n")
        if result["text"] not in final_context:
            final_context.append(result["text"])
    
    
    final_context = "\n".join(final_context[:4])
    # print(final_context)
    return final_context

In [ ]:
import datetime

current_date = datetime.datetime.now()
while True:
    query = input("Enter the query : ")
    payload = {
      "question": query,
      "context": rerank_result(query),
      "company_name": "grandcholan",
      "domain": "foods and services"
    }
    
    response = requests.post("http://46.43.144.145:7030/rule_based_qa",json=payload)
    
    print(response.json())

Enter the query :  how to contact yu


['How to get in touch with Yu', 'Reach out to Yu for communication', 'Contact information for Yu', 'Connect with the person named Yu', 'how to contact yu']
To contact Grand Cholan Indian Restaurant, you can call them at 020 7515 6888 or visit their website at www.grandcholan.com to reserve a table and get in touch with the team through email or online form. They are located at 3 Turnberry Quay, Docklands, London E14 9RD UK and are open from Monday to Sunday between 12pm-3pm and then again from 5pm-10pm for your dining pleasure.


Enter the query :  how to contact you


['How to reach you for assistance', 'Get in touch with me for help', 'Contact information for support', 'Reach out to receive guidance', 'how to contact you']
020 7515 6888
    Email: info@grandcholan.com
    Address: 3 Turnberry Quay, Docklands, London E14 9RD UK


Enter the query :  how to contact you


['Contact Me', 'Reach Out to Me', 'Get in Touch with Me', 'Connect with Me', 'how to contact you']
You can contact us through various methods. Our main phone number is 020 7515 6888, which you can call to make reservations or inquire about our restaurant. You can also email us at info@grandcholan.com for any questions or concerns you may have. Additionally, we have a website where you can reserve a table online at www.grandcholan.com and get more information about our restaurant and menu offerings.


Enter the query :  how to book flight


['How to reserve a flight ticket', 'Flight booking process explained', 'Booking flights made easy', 'Guide for flight reservation', 'how to book flight']
To book a flight, you can follow these steps:
1. Choose your preferred airline or travel website to search for flights. Some popular options include Expedia, Skyscanner, Kayak, and Google Flights.
2. Enter your departure city and destination in the search fields provided on the chosen platform's homepage or main menu. You may also need to specify dates of travel if not already set by default (usually 30 days from today).
3. Review the available flight options displayed with their respective prices and schedules based on your preferences like direct flights vs layovers, number of stops etc., as well as any additional services such as baggage allowance or seat selection that might be offered at an extra cost by some airlines/websites during this process too!
4. Once you have found a suitable flight option that meets all your requirement

In [29]:
import datetime

current_date = datetime.datetime.now()
while True:
    query = input("Enter the query : ")
    payload = {
      "message_content": query,
      "content": "price of idli is £7.5"
    }
    
    response = requests.post("http://46.43.144.145:7030/rule_based_qa",json=payload)
    
    print(response.json())

Enter the query :  how to contact you


KeyError: 'response'

In [ ]:
class HuggingFaceLLM:
    def __init__(self,model_name:str):

In [ ]:
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

model_name_or_path = "TheBloke/Mistral-7B-OpenOrca-AWQ"



class VariationChain:
        
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_name_or_path,
        low_cpu_mem_usage=True,
        device_map="cuda:0"
    )
    
    generation_params = {
        "do_sample": True,
        "temperature": 0.1,
        "top_p": 0.95,
        "top_k": 40,
        "max_new_tokens": 2000,
        "repetition_penalty": 1.1
    }
    
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        **generation_params
    )

In [ ]:
class FusionRetriever:

    def __init__(self,vector_db,reranker = None,top_k:int = 1):
        self.vector_db = vector_db
        self.reranker = reranker
        self.top_k = top_k

    def 